# Project: Wrangling and Analyze Data

## Data Gathering

In [1]:
# imports
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json

In [2]:
# import Twitter API key
import constants

In [3]:
# set max column display width to 260 characters so that we can read full tweet text
pd.options.display.max_colwidth = 260

1. Import the twitter_archive_enhanced.csv

In [4]:
df_ta = pd.read_csv('./source_data/twitter-archive-enhanced.csv')
df_ta.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

   retweeted_status_id  retweeted_status_user_id retweeted_status_timestamp  \
0                  NaN                       NaN                        NaN   
1                  NaN                       NaN                        NaN   
2                  NaN                       NaN                        NaN   
3                  NaN                       NaN                        NaN   
4                  NaN                       NaN                        NaN   

                                                                                                                     expanded_urls  \
0                                                                  https://twitter.com/dog_rates/status/892420643555336193/photo/1   
1                                                                  https://twitter.com/dog_rates/status/892177421306343426/photo/1   
2                                                                  https://twitter.com/dog_rates/status/891815181378084864/photo/1   
3                                                                  https://twitter.com/dog_rates/status/891689557279858688/photo/1   
4  https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1   

   rating_numerator  rating_denominator      name doggo floofer pupper puppo  
0                13                  10   Phineas  None    None   None  None  
1                13                  10     Tilly  None    None   None  None  
2                12                  10    Archie  None    None   None  None  
3                13                  10     Darla  None    None   None  None  
4                12                  10  Franklin  None    None   None  None

2. Use `requests` to download the tweet image predication (image_predictions.tsv)

In [5]:
if not os.path.exists('./source_data'):
    os.makedirs('./source_data/')

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

# confirm request successful
assert r.status_code == 200

# write r.content to output file
with open('./source_data/image_predictions.tsv', 'wb') as fp:
    fp.write(r.content)


In [6]:
# import downloaded file to a second df
df_ip = pd.read_csv('./source_data/image_predictions.tsv', delimiter='\t')
df_ip.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [7]:
# import the tweepy api library
import tweepy

# set up client object with authentication
client = tweepy.Client(
    bearer_token=constants.BEARER_TOKEN_TWITTER,
    wait_on_rate_limit=True
)


In [8]:
# check number of requests we'll be making
len(df_ta)

2356

In [9]:
# NOTE: ** THIS STEP TAKES 100+ MINS TO COMPLETE! **

# # create a path
# if not os.path.exists('./source_data'):
#     os.makedirs('./source_data/')
# # clear the file if it already exists
# if os.path.exists('./source_data/tweet_json.txt'):
#     os.remove('./source_data/tweet_json.txt')
# 
# # loop through tweet_ids in the df_ta file
# additional_data = []
# additional_data_errors = []
# for tweet_id in df_ta['tweet_id']:
#     # hit the twitter api and gather retweet count, like count 
#     r = client.get_tweet(tweet_id, tweet_fields='public_metrics')
#     print(tweet_id)
#     # form the metrics into a dictionary
#     if r.errors == []:
#         public_metrics = {
#             'tweet_id': tweet_id,
#             'retweet_count': r.data.public_metrics['retweet_count'],
#             'reply_count': r.data.public_metrics['reply_count'],
#             'like_count': r.data.public_metrics['like_count'],
#             'quote_count': r.data.public_metrics['quote_count'],
#         }
#         # add the data to the metrics
#         additional_data.append(public_metrics)
#     else:
#         print(tweet_id, r.errors)
#         additional_data_errors.append(r.errors)
# 
# # open the file, write the result
# with open('./source_data/tweet_json.txt', 'a') as fp:
#     # write the list of dicts the file
#     json.dump(additional_data, fp)


In [10]:
# # check how many tweets could not be retrieved
# len(additional_data_errors)

In [11]:
# read the tweet_json into a dataframe
df_count_data = pd.read_json('./source_data/tweet_json.txt')

In [12]:
df_count_data

tweet_id  retweet_count  reply_count  like_count  quote_count
0     892420643555336193           6982          139       33729          219
1     892177421306343426           5284          170       29261          274
2     891815181378084864           3469          110       21996          145
3     891689557279858688           7197          142       36819          286
4     891327558926688256           7731          193       35199          296
...                  ...            ...          ...         ...          ...
2322  666049248165822465             36            9          88            0
2323  666044226329800704            115            1         246            2
2324  666033412701032449             36            1         100            0
2325  666029285002620928             39            0         112            0
2326  666020888022790149            422           20        2286           30

[2327 rows x 5 columns]

## Assessing Data


### Twitter archive

Visual assessment:

In [13]:
df_ta.head(10)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   
5  891087950875897856                    NaN                  NaN   
6  890971913173991426                    NaN                  NaN   
7  890729181411237888                    NaN                  NaN   
8  890609185150312448                    NaN                  NaN   
9  890240255349198849                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   
5  2017-07-29 00:08:17 +0000   
6  2017-07-28 16:27:12 +0000   
7  2017-07-28 00:22:40 +0000   
8  2017-07-27 16:25:51 +0000   
9  2017-07-26 15:59:51 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
5  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
6  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
7  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
8  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
9  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                     text  \
0                                                           This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1      This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                       This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                                 This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4      This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   
5      Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh   
6  Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl   
7                          When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq   
8                      This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b   
9           This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A   

   retweeted_sta

Initial notes:
* Do all of the in_reply_to_ and retweeted_ columns imply that the tweet is a retweet?
* Is timestamp encoded as datetime?
* is 'source' column useful?  It is formatted as an html tag

Programmatic Assessment:

In [14]:
# check a subset of data with in_reply_to_ and retweeted_ columns populated
df_ta[df_ta['retweeted_status_timestamp'].notna()][['text', 'expanded_urls']].sample(5)

text  \
598  RT @dog_rates: Say hello to Jack (pronounced "Kevin"). He's a Virgo Episcopalian. Can summon rainbows. 11/10 magical as hell https://t.co/Y…   
943           RT @dog_rates: HEY PUP WHAT'S THE PART OF THE HUMAN BODY THAT CONNECTS THE FOOT AND THE LEG? 11/10 so smart https://t.co/XQ1tRUmO3z   
450                       RT @dog_rates: This is Seamus. He's very bad at entering pools. Still a very good boy tho 11/10 https://t.co/hfi264QwYM   
829  RT @dog_rates: When it's Janet from accounting's birthday but you can't eat the cake cuz it's chocolate. 10/10 hang in there pupper https:/…   
479  RT @dog_rates: Say hello to Levi. He's a Madagascan Butterbop. One of the more docile Butterbops I've seen. 12/10 would give all the pets h…   

                                                                                                                                                                                                                                                       expanded_urls  
598                                                                                                                                                                                                  https://twitter.com/dog_rates/status/683391852557561860/photo/1  
943                                                                                                                                                                                                      https://vine.co/v/ibvnzrauFuV,https://vine.co/v/ibvnzrauFuV  
450                                                                                                                                                                                                      https://vine.co/v/5QWd3LZqXxd,https://vine.co/v/5QWd3LZqXxd  
829                                                                                                                                                                                                  https://twitter.com/dog_rates/status/700143752053182464/photo/1  
479  https://twitter.com/dog_rates/status/791406955684368384/photo/1,https://twitter.com/dog_rates/status/791406955684368384/photo/1,https://twitter.com/dog_rates/status/791406955684368384/photo/1,https://twitter.com/dog_rates/status/791406955684368384/photo/1

**Observation:** Anything with the retweeted_status_ stamp appears to be a retweet, not an original 

In [15]:
df_ta[df_ta['in_reply_to_status_id'].notna()].sample(5)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
148   863427515083354112           8.634256e+17         7.759620e+07   
1501  692142790915014657           6.920419e+17         4.196984e+09   
1446  696490539101908992           6.964887e+17         4.196984e+09   
611   797165961484890113           7.971238e+17         2.916630e+07   
1842  675870721063669760           6.757073e+17         4.196984e+09   

                      timestamp  \
148   2017-05-13 16:15:35 +0000   
1501  2016-01-27 00:31:15 +0000   
1446  2016-02-08 00:27:39 +0000   
611   2016-11-11 19:55:50 +0000   
1842  2015-12-13 02:51:51 +0000   

                                                                                  source  \
148   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1501  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1446  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
611   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1842  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                  text  \
148             @Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy content above I'm leaning towards 12/10   
1501                                      These are some pictures of Teddy that further justify his 13/10 rating. Please enjoy https://t.co/tDkJAnQsbQ   
1446                                             After reading the comments I may have overestimated this pup. Downgraded to a 1/10. Please forgive me   
611                                                                                            @JODYHiGHROLLER it may be an 11/10 but what do I know 😉   
1842  &amp; this is Yoshi. Another world record contender 11/10 (what the hell is happening why are there so many contenders?) https://t.co/QG708dDNH6   

      retweeted_status_id  retweeted_status_user_id  \
148                   NaN                       NaN   
1501                  NaN                       NaN   
1446                  NaN                       NaN   
611                   NaN                       NaN   
1842                  NaN                       NaN   

     retweeted_status_timestamp  \
148                         NaN   
1501                        NaN   
1446                        NaN   
611                         NaN   
1842                        NaN   

                                                                                                                                                                                                                                                        expanded_urls  \
148                                                                                                                                                                                                                                                               NaN   
1501  https://twitter.com/dog_rates/status/692142790915014657/photo/1,https://twitter.com/dog_rates/status/692142790915014657/photo/1,https://twitter.com/dog_rates/status/692142790915014657/photo/1,https://twitter.com/dog_rates/status/692142790915014657/photo/1   
1446                                                                                                                                                                                                                                                              NaN   
611                                                                                                                                                                                                                                                               NaN   
1842                       

**Observation:** Anything with a non-null in_reply_to_ seems to be a reply, not an original rating.  Most of these don't have appear to have a URL or dog name

In [16]:
# completeness - check number of rows
df_ta.shape

(2356, 17)

In [17]:
# completeness - check for NaN
df_ta.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [18]:
df_ta[df_ta['expanded_urls'].isna()].sample(5)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
427   821153421864615936           8.211526e+17         1.132119e+08   
1905  674606911342424069           6.744689e+17         4.196984e+09   
1844  675849018447167488           6.758457e+17         4.196984e+09   
1605  685681090388975616           6.855479e+17         4.196984e+09   
234   847617282490613760           8.476062e+17         4.196984e+09   

                      timestamp  \
427   2017-01-17 00:33:26 +0000   
1905  2015-12-09 15:09:55 +0000   
1844  2015-12-13 01:25:37 +0000   
1605  2016-01-09 04:34:45 +0000   
234   2017-03-31 01:11:22 +0000   

                                                                                  source  \
427   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1905  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1844  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1605  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
234   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                        text  \
427                                                                                 @imgur for a polar bear tho I'd say 13/10 is appropriate   
1905  The 13/10 also takes into account this impeccable yard. Louis is great but the future dad in me can't ignore that luscious green grass   
1844                         This dog is being demoted to a 9/10 for not wearing a helmet while riding. Gotta stay safe out there. Thank you   
1605      Jack deserves another round of applause. If you missed this earlier today I strongly suggest reading it. Wonderful first 14/10 🐶❤️   
234                                                          .@breaannanicolee PUPDATE: Cannon has a heart on his nose. Pupgraded to a 13/10   

      retweeted_status_id  retweeted_status_user_id  \
427                   NaN                       NaN   
1905                  NaN                       NaN   
1844                  NaN                       NaN   
1605                  NaN                       NaN   
234                   NaN                       NaN   

     retweeted_status_timestamp expanded_urls  rating_numerator  \
427                         NaN           NaN                13   
1905                        NaN           NaN                13   
1844                        NaN           NaN                 9   
1605                        NaN           NaN                14   
234                         NaN           NaN                13   

      rating_denominator  name doggo floofer pupper puppo  
427                   10  None  None    None   None  None  
1905                  10  None  None    None   None  None  
1844                  10  None  None    None   None  None  
1605                  10  None  None    None   None  None  
234                   10  None  None    None   None  None

Observations:
* Only <100 of the in_reply_to_ columns are non-null
* ~150 of the retweeted_status_id columns are non-null
* 59 expanded urls are null - but these seem to be only replies

In [19]:
# completeness - check for duplicates
df_ta['tweet_id'].duplicated().sum()

0

In [20]:
# it looks like some dog names are None - confirm this
df_ta['name'].value_counts()

None          745
a              55
Charlie        12
Cooper         11
Lucy           11
             ... 
Dex             1
Ace             1
Tayzie          1
Grizzie         1
Christoper      1
Name: name, Length: 957, dtype: int64

In [21]:
# check if these names are truly unknown or if this was an error reading the tweet text
df_ta[df_ta['name']=='None']['text'].sample(10)

1896                                                                                                  "🎶 DO YOU BELIEVE IN LIFE AFTER LOVE 🎶"\n11/10 https://t.co/URNs5zFskc
1380                                                                 I'm not sure what's happening here, but it's pretty spectacular. 12/10 for both https://t.co/JKXh0NbBNL
386                             RT @dog_rates: Please only send in dogs. We only rate dogs, not seemingly heartbroken ewoks. Thank you... still 10/10 would console https:/…
1042                             When you're given AUX cord privileges from the back seat and accidentally start blasting an audiobook... both 10/10 https://t.co/gCCrY8P0K9
912                             Here's another picture without a dog in it. Idk why you guys keep sending these. 4/10 just because that's a neat rug https://t.co/mOmnL19Wsl
2339                                        Oh my. Here you are seeing an Adobe Setter giving birth to twins!!! The world is an amazing

In [22]:
# check if these names are truly unknown or if this was an error reading the tweet text
df_ta[df_ta['name']=='a']['text'].sample(10)

2347                    My oh my. This is a rare blond Canadian terrier on wheels. Only $8.98. Rather docile. 9/10 very rare https://t.co/yWBqbrzy8O
1193     People please. This is a Deadly Mediterranean Plop T-Rex. We only rate dogs. Only send in dogs. Thanks you... 11/10 https://t.co/2ATDsgHD4n
2128      This is a Speckled Cauliflower Yosemite named Hemry. He's terrified of intruder dog. Not one bit comfortable. 9/10 https://t.co/yV3Qgjh8iN
2327     This is a southern Vesuvius bumblegruff. Can drive a truck (wow). Made friends with 5 other nifty dogs (neat). 7/10 https://t.co/LopTBkKa8h
1853    This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI
1340                              Here is a heartbreaking scene of an incredible pupper being laid to rest. 10/10 RIP pupper https://t.co/81mvJ0rGRu
2034        This is a Tuscaloosa Alcatraz named Jacob (Yacōb). Loves to sit in swing. Stellar tongue. 11/1

In [23]:
# check if these names are truly unknown or if this was an error reading the tweet text
df_ta[df_ta['name']=='the']['text']

1527                                   Stop sending in lobsters. This is the final warning. We only rate dogs. Thank you... 9/10 https://t.co/B9ZXXKJYNx
1603        This is the newly formed pupper a capella group. They're just starting out but I see tons of potential. 8/10 for all https://t.co/wbAcvFoNtn
1797                                                    This is the happiest pupper I've ever seen. 10/10 would trade lives with https://t.co/ep8ATEJwRb
1815                                                          This is the saddest/sweetest/best picture I've been sent. 12/10 😢🐶 https://t.co/vQ2Lw1BLBF
2037    This is the best thing I've ever seen so spread it like wildfire &amp; maybe we'll find the genius who created it. 13/10 https://t.co/q6RsuOVYwU
2212              Never forget this vine. You will not stop watching for at least 15 minutes. This is the second coveted.. 13/10 https://t.co/roqIxCvEB3
2345                                         This is the happiest dog you will eve

**Observations:** 
* Some dog names are 'None', 'a' and 'the'
* The 'None' names seem correct - the dogs do not have a name
* The 'a' names sometimes contain a name (generally after the word 'named')
* The 'the' names should all be 'None' 

In [24]:
# check format of 'source' column
df_ta['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

**Observations:** This seems to actually be a categorical with the sources being: Twitter for iPhone; Vine; Twitter Web Client; TweetDeck

In [25]:
# check format of 'expanded_url' column
df_ta['expanded_urls'].value_counts()

https://twitter.com/dog_rates/status/667152164079423490/photo/1                                                                                                                                                                                                    2
https://twitter.com/dog_rates/status/718631497683582976/photo/1                                                                                                                                                                                                    2
https://twitter.com/dog_rates/status/783839966405230592/photo/1,https://twitter.com/dog_rates/status/783839966405230592/photo/1,https://twitter.com/dog_rates/status/783839966405230592/photo/1                                                                    2
https://twitter.com/dog_rates/status/774314403806253056/photo/1,https://twitter.com/dog_rates/status/774314403806253056/photo/1,https://twitter.com/dog_rates/status/774314403806253056/photo/1,https://twitter.com/dog_r

**Observations:** Some URLs contain repeats of the same URL info

In [26]:
# Validity: check data types
df_ta.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
doggo                          object
floofer                        object
pupper                         object
puppo                          object
dtype: object

**Observations:** 
* timestamp is string but should be datetime
* doggo, floofer, pupper, puppo are encoded as strings but should be boolean (true / false)

In [27]:
# check if doggo, floofer, pupper and puppo are always mutually exclusive
((df_ta['doggo']=='doggo') & df_ta['floofer']=='floofer').sum() + \
((df_ta['doggo']=='doggo') & df_ta['pupper']=='pupper').sum() + \
((df_ta['doggo']=='doggo') & df_ta['puppo']=='puppo').sum() + \
((df_ta['floofer']=='floofer') & df_ta['pupper']=='pupper').sum() + \
((df_ta['floofer']=='floofer') & df_ta['puppo']=='puppo').sum() + \
((df_ta['pupper']=='pupper') & df_ta['puppo']=='puppo').sum()

0

**Observations:** 
* doggo, floofer, pupper and puppo are mutually exclusive => it would make most sense to encode them as a single column with a categorical

In [28]:
# check summary of numerical datatypes
df_ta.describe()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
count  2.356000e+03           7.800000e+01         7.800000e+01   
mean   7.427716e+17           7.455079e+17         2.014171e+16   
std    6.856705e+16           7.582492e+16         1.252797e+17   
min    6.660209e+17           6.658147e+17         1.185634e+07   
25%    6.783989e+17           6.757419e+17         3.086374e+08   
50%    7.196279e+17           7.038708e+17         4.196984e+09   
75%    7.993373e+17           8.257804e+17         4.196984e+09   
max    8.924206e+17           8.862664e+17         8.405479e+17   

       retweeted_status_id  retweeted_status_user_id  rating_numerator  \
count         1.810000e+02              1.810000e+02       2356.000000   
mean          7.720400e+17              1.241698e+16         13.126486   
std           6.236928e+16              9.599254e+16         45.876648   
min           6.661041e+17              7.832140e+05          0.000000   
25%           7.186315e+17              4.196984e+09         10.000000   
50%           7.804657e+17              4.196984e+09         11.000000   
75%           8.203146e+17              4.196984e+09         12.000000   
max           8.874740e+17              7.874618e+17       1776.000000   

       rating_denominator  
count         2356.000000  
mean            10.455433  
std              6.745237  
min              0.000000  
25%             10.000000  
50%             10.000000  
75%             10.000000  
max            170.000000

In [29]:
# check rating_denominator < 10
df_ta[df_ta['rating_denominator']<10]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
313   835246439529840640           8.352460e+17           26259576.0   
516   810984652412424192                    NaN                  NaN   
2335  666287406224695296                    NaN                  NaN   

                      timestamp  \
313   2017-02-24 21:54:03 +0000   
516   2016-12-19 23:06:23 +0000   
2335  2015-11-16 16:11:11 +0000   

                                                                                  source  \
313   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
516   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2335  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                           text  \
313                                                                          @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho   
516   Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx   
2335                                   This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv   

      retweeted_status_id  retweeted_status_user_id  \
313                   NaN                       NaN   
516                   NaN                       NaN   
2335                  NaN                       NaN   

     retweeted_status_timestamp  \
313                         NaN   
516                         NaN   
2335                        NaN   

                                                                                            expanded_urls  \
313                                                                                                   NaN   
516   https://www.gofundme.com/sams-smile,https://twitter.com/dog_rates/status/810984652412424192/photo/1   
2335                                      https://twitter.com/dog_rates/status/666287406224695296/photo/1   

      rating_numerator  rating_denominator  name doggo floofer pupper puppo  
313                960                   0  None  None    None   None  None  
516                 24                   7   Sam  None    None   None  None  
2335                 1                   2    an  None    None   None  None

In [30]:
# check tweet numerator > 20
df_ta[(df_ta['rating_numerator']>20) & (df_ta['in_reply_to_status_id'].isna()) & (df_ta['retweeted_status_id'].isna()) ]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
433   820690176645140481                    NaN                  NaN   
516   810984652412424192                    NaN                  NaN   
695   786709082849828864                    NaN                  NaN   
763   778027034220126208                    NaN                  NaN   
902   758467244762497024                    NaN                  NaN   
979   749981277374128128                    NaN                  NaN   
1120  731156023742988288                    NaN                  NaN   
1202  716439118184652801                    NaN                  NaN   
1228  713900603437621249                    NaN                  NaN   
1254  710658690886586372                    NaN                  NaN   
1274  709198395643068416                    NaN                  NaN   
1351  704054845121142784                    NaN                  NaN   
1433  697463031882764288                    NaN                  NaN   
1635  684222868335505415                    NaN                  NaN   
1712  680494726643068929                    NaN                  NaN   
1779  677716515794329600                    NaN                  NaN   
1843  675853064436391936                    NaN                  NaN   
2074  670842764863651840                    NaN                  NaN   

                      timestamp  \
433   2017-01-15 17:52:40 +0000   
516   2016-12-19 23:06:23 +0000   
695   2016-10-13 23:23:56 +0000   
763   2016-09-20 00:24:34 +0000   
902   2016-07-28 01:00:57 +0000   
979   2016-07-04 15:00:45 +0000   
1120  2016-05-13 16:15:54 +0000   
1202  2016-04-03 01:36:11 +0000   
1228  2016-03-27 01:29:02 +0000   
1254  2016-03-18 02:46:49 +0000   
1274  2016-03-14 02:04:08 +0000   
1351  2016-02-28 21:25:30 +0000   
1433  2016-02-10 16:51:59 +0000   
1635  2016-01-05 04:00:18 +0000   
1712  2015-12-25 21:06:00 +0000   
1779  2015-12-18 05:06:23 +0000   
1843  2015-12-13 01:41:41 +0000   
2074  2015-11-29 05:52:33 +0000   

                                                                                   source  \
433    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
516    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
695    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
763    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
902    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
979   <a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>   
1120   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1202   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1228   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1254   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1274   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1351   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1433   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1635   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1712   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1779   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1843   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2074   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                           text  \
433                               

**Observations:** 
* tweet 810984652412424192 should be dropped - not a rating
* tweet 666287406224695296 rating should be corrected to 9/10 (incorrectly picked up as 1/2)
* rating_numerators over 20 look correct

In [31]:
df_ta['timestamp'].max(), df_ta['timestamp'].min()

('2017-08-01 16:23:56 +0000', '2015-11-15 22:32:08 +0000')

### Tweet Image Prediction

Visual assessment:

In [32]:
# visual assessment
df_ip.head(10)

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5  666050758794694657  https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
6  666051853826850816  https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
7  666055525042405380  https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
8  666057090499244032  https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
9  666058600524156928  https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   
5        1    Bernese_mountain_dog  0.651137    True    English_springer   
6        1              box_turtle  0.933012   False          mud_turtle   
7        1                    chow  0.692517    True     Tibetan_mastiff   
8        1           shopping_cart  0.962465   False     shopping_basket   
9        1        miniature_poodle  0.201493    True            komondor   

    p2_conf  p2_dog                           p3   p3_conf  p3_dog  
0  0.156665    True            Shetland_sheepdog  0.061428    True  
1  0.074192    True          Rhodesian_ridgeback  0.072010    True  
2  0.138584    True                   bloodhound  0.116197    True  
3  0.360687    True           miniature_pinscher  0.222752    True  
4  0.243682    True                     Doberman  0.154629    True  
5  0.263788    True   Greater_Swiss_Mountain_dog  0.016199    True  
6  0.045885   False                     terrapin  0.017885   False  
7  0.058279    True                     fur_coat  0.054449   False  
8  0.014594   False             golden_retriever  0.007959    True  
9  0.192305    True  soft-coated_wheaten_terrier  0.082086    True

**Observations:** Structurally, this is information about the dog, so should live with the other info about the dog in the tweet archive data

Programmatic Assessment:

In [33]:
# check for number of records
df_ip.shape

(2075, 12)

**Observations:** We have fewer records here than in the tweet archive

In [34]:
# check for NaN
df_ip.isna().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [35]:
# check for duplicates
df_ip['tweet_id'].duplicated().sum()

0

In [36]:
# check datatypes
df_ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [37]:
# check descriptive stats of numerical fields
df_ip.describe()

tweet_id      img_num      p1_conf       p2_conf       p3_conf
count  2.075000e+03  2075.000000  2075.000000  2.075000e+03  2.075000e+03
mean   7.384514e+17     1.203855     0.594548  1.345886e-01  6.032417e-02
std    6.785203e+16     0.561875     0.271174  1.006657e-01  5.090593e-02
min    6.660209e+17     1.000000     0.044333  1.011300e-08  1.740170e-10
25%    6.764835e+17     1.000000     0.364412  5.388625e-02  1.622240e-02
50%    7.119988e+17     1.000000     0.588230  1.181810e-01  4.944380e-02
75%    7.932034e+17     1.000000     0.843855  1.955655e-01  9.180755e-02
max    8.924206e+17     4.000000     1.000000  4.880140e-01  2.734190e-01

### Tweet count data

Visual assessment:

In [38]:
# visual inspection
df_count_data.head()

tweet_id  retweet_count  reply_count  like_count  quote_count
0  892420643555336193           6982          139       33729          219
1  892177421306343426           5284          170       29261          274
2  891815181378084864           3469          110       21996          145
3  891689557279858688           7197          142       36819          286
4  891327558926688256           7731          193       35199          296

Programmatic Assessment:

In [39]:
# check for completeness
df_count_data.shape

(2327, 5)

**Observations:** We have some missing info - there were some missing id errors when fetching from Tweepy so it is likely from those

In [40]:
# completness - check for NaN
df_count_data.isna().sum()

tweet_id         0
retweet_count    0
reply_count      0
like_count       0
quote_count      0
dtype: int64

In [41]:
# check data types
df_count_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   tweet_id       2327 non-null   int64
 1   retweet_count  2327 non-null   int64
 2   reply_count    2327 non-null   int64
 3   like_count     2327 non-null   int64
 4   quote_count    2327 non-null   int64
dtypes: int64(5)
memory usage: 91.0 KB


In [42]:
# check descriptive stats
df_count_data.describe()

tweet_id  retweet_count  reply_count     like_count  quote_count
count  2.327000e+03    2327.000000  2327.000000    2327.000000  2327.000000
mean   7.417930e+17    2464.064890    42.185217    7034.757628    71.328320
std    6.820795e+16    4172.215212    70.788044   10930.969939   220.257932
min    6.660209e+17       1.000000     0.000000       0.000000     0.000000
25%    6.781394e+17     493.500000     9.000000    1222.000000     0.000000
50%    7.178418e+17    1148.000000    23.000000    3043.000000     1.000000
75%    7.986547e+17    2851.000000    48.000000    8574.000000    69.500000
max    8.924206e+17   70496.000000  1121.000000  144444.000000  4347.000000

In [43]:
df_count_data['tweet_id'].duplicated().sum()

0

## Quality issues
Completeness:

1. df_ta includes retweets and replies (indicated by non-null in_reply_to and retweeted_ columns) - **done**
2. Dog named 'a' should either be 'None' or their actual name - **done**
3. Dogs named 'the' should be 'None'
4. We do not have retweet / like data for all tweets - **done**
5. We do not have image predictions for all tweets - **not addressable: we don't have access to image classification engine**
6. Some expanded urls are missing - **done**

Validity:

7. Source is encoded as an HTML tag, but is actually a categorical - **done**
8. Expanded urls generally contain repeats of the same url separated by a comma - **done**
9. Timestamp is string but should be datetime - **done**
10. Doggo, floofer, etc are encoded as string - **done**

Accuracy:

11. tweet 810984652412424192 should be dropped - not a rating - **done**
12. tweet 666287406224695296 rating should be corrected to 9/10 (incorrectly picked up as 1/2) - **done**

## Tidiness issues
1. Doggo, floofer, pupper and puppo should be encoded as a single categorical column - **done**
2. The tweet archive contains information both about the dog (e.g. the dog’s name) and the tweet (e.g. the source of the tweet).  These should be separated into two separate tables **done**

## Cleaning

### Copy data before cleaning

In [44]:
# copy dfs before cleaning
df_ta_clean = df_ta.copy(deep=True)
df_ip_clean = df_ip.copy(deep=True)
df_cd_clean = df_count_data.copy(deep=True)

### Issue: Tweet archive data contains retweets and replies

**Define:** 
* Filter any rows which have non-null values for retweeted_ or in_reply_to and remove from dataframe
* Remove the retweeted_ and reply_to_ columns

**Code:**

In [45]:
df_ta_clean.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

   retweeted_status_id  retweeted_status_user_id retweeted_status_timestamp  \
0                  NaN                       NaN                        NaN   
1                  NaN                       NaN                        NaN   
2                  NaN                       NaN                        NaN   
3                  NaN                       NaN                        NaN   
4                  NaN                       NaN                        NaN   

                                                                                                                     expanded_urls  \
0                                                                  https://twitter.com/dog_rates/status/892420643555336193/photo/1   
1                                                                  https://twitter.com/dog_rates/status/892177421306343426/photo/1   
2                                                                  https://twitter.com/dog_rates/status/891815181378084864/photo/1   
3                                                                  https://twitter.com/dog_rates/status/891689557279858688/photo/1   
4  https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1   

   rating_numerator  rating_denominator      name doggo floofer pupper puppo  
0                13                  10   Phineas  None    None   None  None  
1                13                  10     Tilly  None    None   None  None  
2                12                  10    Archie  None    None   None  None  
3                13                  10     Darla  None    None   None  None  
4                12                  10  Franklin  None    None   None  None

In [46]:
# filter and remove any columns with non-null values for in_reply_to_
df_ta_clean = df_ta_clean[(df_ta_clean['in_reply_to_status_id'].isna()) & (df_ta_clean['in_reply_to_user_id'].isna())]

In [47]:
# filter and remove any columns with non-null values for in_reply_to_
df_ta_clean = df_ta_clean[(df_ta_clean['retweeted_status_id'].isna()) & (df_ta_clean['retweeted_status_user_id'].isna()) & (df_ta_clean['retweeted_status_timestamp'].isna())]

In [48]:
# confirm this worked
print(
    df_ta_clean['in_reply_to_status_id'].notna().sum(),
    df_ta_clean['in_reply_to_user_id'].notna().sum(),
    df_ta_clean['retweeted_status_id'].notna().sum(),
    df_ta_clean['retweeted_status_user_id'].notna().sum(),
    df_ta_clean['retweeted_status_timestamp'].notna().sum(),
)


0 0 0 0 0


**Test:**

In [49]:
# confirm this worked
df_ta_clean.shape

(2097, 17)

In [50]:
# drop the retweet and reply columns
df_ta_clean.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], inplace=True)

In [51]:
df_ta_clean.head()

tweet_id                  timestamp  \
0  892420643555336193  2017-08-01 16:23:56 +0000   
1  892177421306343426  2017-08-01 00:17:27 +0000   
2  891815181378084864  2017-07-31 00:18:03 +0000   
3  891689557279858688  2017-07-30 15:58:51 +0000   
4  891327558926688256  2017-07-29 16:00:24 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

                                                                                                                     expanded_urls  \
0                                                                  https://twitter.com/dog_rates/status/892420643555336193/photo/1   
1                                                                  https://twitter.com/dog_rates/status/892177421306343426/photo/1   
2                                                                  https://twitter.com/dog_rates/status/891815181378084864/photo/1   
3                                                                  https://twitter.com/dog_rates/status/891689557279858688/photo/1   
4  https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1   

   rating_numerator  rating_denominator      name doggo floofer pupper puppo  
0                13                  10   Phineas  None    None   None  None  
1                13                  10     Tilly  None    None   None  None  
2                12                  10    Archie  None    None   None  None  
3                13                  10     Darla  None    None   None  None  
4                12                  10  Franklin  None    None   None  None

### Issue: The doggo, floofer, pupper, puppo columns should be one column for a single categorical variable

Define
* Create a new column 'dog_type' and initialize the value to 'None' for every row
* Then populate dog_type when any of doggo, floofer, pupper, puppo columns are not 'None'
* Cast the new column to type categorical
* Delete the original 4 columns

**Code:**

In [52]:
df_ta_clean['dog_type'] = 'None'
df_ta_clean.loc[df_ta_clean['doggo']!='None', 'dog_type'] = 'doggo'
df_ta_clean.loc[df_ta_clean['floofer']!='None', 'dog_type'] = 'floofer'
df_ta_clean.loc[df_ta_clean['pupper']!='None', 'dog_type'] = 'pupper'
df_ta_clean.loc[df_ta_clean['puppo']!='None', 'dog_type'] = 'puppo'


In [53]:
# confirm this worked
df_ta_clean[df_ta_clean['dog_type']=='None'].sample(5)

tweet_id                  timestamp  \
1881  675003128568291329  2015-12-10 17:24:21 +0000   
1292  708109389455101952  2016-03-11 01:56:49 +0000   
1371  702276748847800320  2016-02-23 23:39:59 +0000   
870   761672994376806400  2016-08-05 21:19:27 +0000   
144   863907417377173506  2017-05-15 00:02:33 +0000   

                                                                                  source  \
1881  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1292  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1371  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
870   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
144   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                               text  \
1881   Meet Laela. She's adorable. Magnificent eyes. But I don't see a seat belt. Insubordinate.. and churlish. Still 12/10 https://t.co/pCGDgLkLo6   
1292          This is Remington. He was caught off guard by the magical floating cheese. Spooked af. 10/10 deep breaths pup https://t.co/mhPSADiJmZ   
1371                                                         "AND IIIIIIIIIIIEIIIIIIIIIIIII WILL ALWAYS LOVE YOUUUUU" 11/10 https://t.co/rSNCEiTtfI   
870        Ohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboy. 10/10 for all (by happytailsresort) https://t.co/EY8kEFuzK7   
144   This is Albus. He's quite impressive at hide and seek. Knows he's been found this time. 13/10 usually elusive as h*ck https://t.co/ht47njyZ64   

                                                                                                                        expanded_urls  \
1881  https://twitter.com/dog_rates/status/675003128568291329/photo/1,https://twitter.com/dog_rates/status/675003128568291329/photo/1   
1292                                                                  https://twitter.com/dog_rates/status/708109389455101952/photo/1   
1371                                                                  https://twitter.com/dog_rates/status/702276748847800320/photo/1   
870                                                                   https://twitter.com/dog_rates/status/761672994376806400/video/1   
144   https://twitter.com/dog_rates/status/863907417377173506/photo/1,https://twitter.com/dog_rates/status/863907417377173506/photo/1   

      rating_numerator  rating_denominator       name doggo floofer pupper  \
1881                12                  10      Laela  None    None   None   
1292                10                  10  Remington  None    None   None   
1371                11                  10       None  None    None   None   
870                 10                  10       None  None    None   None   
144                 13                  10      Albus  None    None   None   

     puppo dog_type  
1881  None     None  
1292  None     None  
1371  None     None  
870   None     None  
144   None     None

In [54]:
# confirm this worked
df_ta_clean[df_ta_clean['dog_type']!='None'].sample(5)

tweet_id                  timestamp  \
1822  676575501977128964  2015-12-15 01:32:24 +0000   
1705  680836378243002368  2015-12-26 19:43:36 +0000   
1531  690015576308211712  2016-01-21 03:38:27 +0000   
947   752519690950500352  2016-07-11 15:07:30 +0000   
1160  724004602748780546  2016-04-23 22:38:43 +0000   

                                                                                  source  \
1822  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1705  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1531  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
947   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1160  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                            text  \
1822                        This pupper is very passionate about Christmas. Wanted to give the tree a hug. So cute. 8/10 https://t.co/NsGyECJuq7   
1705                                                   This is Ellie. She's secretly ferocious. 12/10 very deadly pupper https://t.co/BF4BW8LUgb   
1531  This pupper can only sleep on shoes. It's a crippling disease. Tearing his family apart. 12/10 I'd totally pet tho https://t.co/03XlvS8izg   
947                              Hopefully this puppo on a swing will help get you through your Monday. 11/10 would push https://t.co/G54yClasz2   
1160                                              This is Luther. He saw a ghost. Spooked af. 11/10 hang in there pupper https://t.co/EdKG43VvEl   

                                                                                                                                                                                        expanded_urls  \
1822                                                                                                                                  https://twitter.com/dog_rates/status/676575501977128964/photo/1   
1705  https://twitter.com/dog_rates/status/680836378243002368/photo/1,https://twitter.com/dog_rates/status/680836378243002368/photo/1,https://twitter.com/dog_rates/status/680836378243002368/photo/1   
1531                                                                  https://twitter.com/dog_rates/status/690015576308211712/photo/1,https://twitter.com/dog_rates/status/690015576308211712/photo/1   
947   https://twitter.com/dog_rates/status/752519690950500352/photo/1,https://twitter.com/dog_rates/status/752519690950500352/photo/1,https://twitter.com/dog_rates/status/752519690950500352/photo/1   
1160  https://twitter.com/dog_rates/status/724004602748780546/photo/1,https://twitter.com/dog_rates/status/724004602748780546/photo/1,https://twitter.com/dog_rates/status/724004602748780546/photo/1   

      rating_numerator  rating_denominator    name doggo floofer  pupper  \
1822                 8                  10    None  None    None  pupper   
1705                12                  10   Ellie  None    None  pupper   
1531                12                  10    None  None    None  pupper   
947                 11                  10    None  None    None    None   
1160                11                  10  Luther  None    None  pupper   

      puppo dog_type  
1822   None   pupper  
1705   None   pupper  
1531   None   pupper  
947   puppo    puppo  
1160   None   pupper

In [55]:
# drop the original columns
df_ta_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

In [56]:
# confirm this worked
df_ta_clean.head()

tweet_id                  timestamp  \
0  892420643555336193  2017-08-01 16:23:56 +0000   
1  892177421306343426  2017-08-01 00:17:27 +0000   
2  891815181378084864  2017-07-31 00:18:03 +0000   
3  891689557279858688  2017-07-30 15:58:51 +0000   
4  891327558926688256  2017-07-29 16:00:24 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

                                                                                                                     expanded_urls  \
0                                                                  https://twitter.com/dog_rates/status/892420643555336193/photo/1   
1                                                                  https://twitter.com/dog_rates/status/892177421306343426/photo/1   
2                                                                  https://twitter.com/dog_rates/status/891815181378084864/photo/1   
3                                                                  https://twitter.com/dog_rates/status/891689557279858688/photo/1   
4  https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1   

   rating_numerator  rating_denominator      name dog_type  
0                13                  10   Phineas     None  
1                13                  10     Tilly     None  
2                12                  10    Archie     None  
3                13                  10     Darla     None  
4                12                  10  Franklin     None

In [57]:
# cast dog_type to category
df_ta_clean['dog_type'] = df_ta_clean['dog_type'].astype('category')

**Test:**

In [58]:
# confirm this worked
df_ta_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tweet_id            2097 non-null   int64   
 1   timestamp           2097 non-null   object  
 2   source              2097 non-null   object  
 3   text                2097 non-null   object  
 4   expanded_urls       2094 non-null   object  
 5   rating_numerator    2097 non-null   int64   
 6   rating_denominator  2097 non-null   int64   
 7   name                2097 non-null   object  
 8   dog_type            2097 non-null   category
dtypes: category(1), int64(3), object(5)
memory usage: 149.7+ KB


### Issue: Information about the dog should go in one table, and information about the tweet's stats should go in another

Define:
* merge the columns with tweet metadata into the count_data df
* remove the columns from the tweet archive df
* rename the dataframes to dog_ratings and tweet_stats
* add a unique id column for the dog_id to the primary key

**Code:**

In [59]:
# merge the columns from df_ta_clean into tweet_stats to form the tweet_stats.  
# note we are doing a right join - we don't want to keep tweet count data for rows we have dropped from df_ta_clean
tweet_stats = df_cd_clean.merge(right=df_ta_clean[['tweet_id', 'timestamp', 'source', 'expanded_urls']], on='tweet_id', how='right')

In [60]:
# unload the df_cd_clean df from memory
del df_cd_clean

In [61]:
# check for NaNs
tweet_stats.isna().sum()

tweet_id         0
retweet_count    8
reply_count      8
like_count       8
quote_count      8
timestamp        0
source           0
expanded_urls    3
dtype: int64

In [62]:
# drop columns from the df_ta_clean
df_ta_clean.drop(columns=['timestamp', 'source', 'expanded_urls'], inplace=True)

In [63]:
# add a dog_id index column to the df_ta_clean dataframe
df_ta_clean.insert(0, 'dog_id', range(1, 1+len(df_ta_clean)))

In [64]:
# rename df_ta_clean to dog_ratings
dog_ratings = df_ta_clean.copy()
del df_ta_clean 

**Test:**

In [65]:
# confirm this worked
dog_ratings.head(5)

dog_id            tweet_id  \
0       1  892420643555336193   
1       2  892177421306343426   
2       3  891815181378084864   
3       4  891689557279858688   
4       5  891327558926688256   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

   rating_numerator  rating_denominator      name dog_type  
0                13                  10   Phineas     None  
1                13                  10     Tilly     None  
2                12                  10    Archie     None  
3                13                  10     Darla     None  
4                12                  10  Franklin     None

In [66]:
# confirm this worked
tweet_stats.sample(5)

tweet_id  retweet_count  reply_count  like_count  quote_count  \
739   752568224206688256         2079.0         39.0      5103.0        118.0   
1522  678708137298427904         2183.0         42.0      5112.0         43.0   
84    873580283840344065         3337.0         56.0     21334.0        102.0   
1962  668507509523615744           87.0          8.0       279.0          0.0   
983   716730379797970944          397.0         17.0      1133.0          0.0   

                      timestamp  \
739   2016-07-11 18:20:21 +0000   
1522  2015-12-20 22:46:44 +0000   
84    2017-06-10 16:39:04 +0000   
1962  2015-11-22 19:13:05 +0000   
983   2016-04-03 20:53:33 +0000   

                                                                                  source  \
739                      <a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>   
1522                     <a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>   
84    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1962  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
983   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                        expanded_urls  
739                                     https://vine.co/v/5W0bdhEUUVT  
1522                                    https://vine.co/v/eQjxxYaQ60K  
84    https://twitter.com/dog_rates/status/873580283840344065/photo/1  
1962  https://twitter.com/dog_rates/status/668507509523615744/photo/1  
983     https://twitter.com/chpsanfrancisco/status/716637124322177024

### Issue: Dogs named 'a' should be their actual name or 'None'; dogs named 'the' should be 'None'

Define:
* Extract names following 'named' or 'name is' where it is present and set as 'dog_name'
* Replace remaining dog_names of 'a' with 'None'

**Code:**

In [67]:
# check if dogs named 'a' actually have a name in the tweet text
dog_ratings[(dog_ratings['name'] == 'a')]['text']

56      Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF
649                                Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq
801                    Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn
1002                   This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW
1004                     Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j14N5R
1017                         This is a carrot. We only rate dogs. Please only send in dogs. You all really should know this by now ...11/10 https://t.co/9e48aPrBm2
1049            

In [68]:
# extract names following 'named' or 'name is' and update `name`
dog_ratings.loc[dog_ratings['name']=='a', 'name'] = dog_ratings[dog_ratings['name']=='a']['text'].str.extract('named?\s?i?s?\s([A-Z][a-z]+)')

In [69]:
# replace the resulting NaNs with 'None'
dog_ratings.loc[dog_ratings['name'].isna(), 'name'] = 'None'

In [70]:
# replace 'the' names with 'None'ArithmeticError
dog_ratings.loc[dog_ratings['name']=='the', 'name'] = 'None'

**Test:**

In [71]:
# confirm this worked
dog_ratings['name'].value_counts()

None          666
Lucy           11
Charlie        11
Oliver         10
Cooper         10
             ... 
Lenox           1
Burt            1
Harvey          1
Blanket         1
Christoper      1
Name: name, Length: 953, dtype: int64

In [72]:
dog_ratings[dog_ratings['name']=='the']

Empty DataFrame
Columns: [dog_id, tweet_id, text, rating_numerator, rating_denominator, name, dog_type]
Index: []

### Issue: Missing retweet / like data


Define:
* Check the date range we are missing retweet / like data for
* either fill or drop rows with missing data

**Code:**

In [73]:
tweet_stats.isna().sum()

tweet_id         0
retweet_count    8
reply_count      8
like_count       8
quote_count      8
timestamp        0
source           0
expanded_urls    3
dtype: int64

In [74]:
# get the ids for some missing tweet stats to check them
tweet_stats[tweet_stats['reply_count'].isna()]

tweet_id  retweet_count  reply_count  like_count  quote_count  \
90    872261713294495745            NaN          NaN         NaN          NaN   
202   844704788403113984            NaN          NaN         NaN          NaN   
234   837366284874571778            NaN          NaN         NaN          NaN   
284   829374341691346946            NaN          NaN         NaN          NaN   
575   779123168116150273            NaN          NaN         NaN          NaN   
687   759923798737051648            NaN          NaN         NaN          NaN   
727   754011816964026368            NaN          NaN         NaN          NaN   
1488  680055455951884288            NaN          NaN         NaN          NaN   

                      timestamp  \
90    2017-06-07 01:19:32 +0000   
202   2017-03-23 00:18:10 +0000   
234   2017-03-02 18:17:34 +0000   
284   2017-02-08 17:00:26 +0000   
575   2016-09-23 01:00:13 +0000   
687   2016-08-01 01:28:46 +0000   
727   2016-07-15 17:56:40 +0000   
1488  2015-12-24 16:00:30 +0000   

                                                                                   source  \
90     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
202    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
234    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
284    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
575    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
687    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
727    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1488  <a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>   

                                                                                                                        expanded_urls  
90    https://twitter.com/dog_rates/status/872261713294495745/photo/1,https://twitter.com/dog_rates/status/872261713294495745/photo/1  
202                                                                   https://twitter.com/dog_rates/status/844704788403113984/photo/1  
234                                                                   https://twitter.com/dog_rates/status/837366284874571778/photo/1  
284   https://twitter.com/dog_rates/status/829374341691346946/photo/1,https://twitter.com/dog_rates/status/829374341691346946/photo/1  
575                                                                   https://twitter.com/dog_rates/status/779123168116150273/photo/1  
687                                                                   https://twitter.com/dog_rates/status/759923798737051648/photo/1  
727   https://twitter.com/dog_rates/status/754011816964026368/photo/1,https://twitter.com/dog_rates/status/754011816964026368/photo/1  
1488                                                                  https://twitter.com/dog_rates/status/680055455951884288/photo/1

In [75]:
# manually checking these, they are missing on twitter, so drop these rows
tweet_stats.drop(tweet_stats[tweet_stats['reply_count'].isna()].index, axis=0, inplace=True)

**Test:**

In [76]:
# confirm this worked
tweet_stats.shape

(2089, 8)

In [77]:
tweet_stats.isna().sum()

tweet_id         0
retweet_count    0
reply_count      0
like_count       0
quote_count      0
timestamp        0
source           0
expanded_urls    3
dtype: int64

### Issue: Tweet source is encoded as an HTML tag, but should be a categorical

Define: 
* use `df.extract()` to extract the part of the string between the tags
* use `.astype()` to convert to category

**Code:**

In [78]:
tweet_stats['source'].unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [79]:
# extract the source from the HTML tag
tweet_stats['source'] = tweet_stats['source'].str.extract('>([A-Za-z-\s]+)<')

In [80]:
tweet_stats['source'].value_counts()

Twitter for iPhone     1957
Vine - Make a Scene      91
Twitter Web Client       31
TweetDeck                10
Name: source, dtype: int64

In [81]:
# Truncate the 'Vine' source
tweet_stats['source'] = tweet_stats['source'].str.replace(pat='Vine - Make a Scene', repl='Vine')

In [82]:
# convert to category type
tweet_stats['source'] = tweet_stats['source'].astype('category')

**Test:**

In [83]:
# confirm this worked
tweet_stats.dtypes

tweet_id            int64
retweet_count     float64
reply_count       float64
like_count        float64
quote_count       float64
timestamp          object
source           category
expanded_urls      object
dtype: object

In [84]:
# confirm this worked
tweet_stats['source'].value_counts()

Twitter for iPhone    1957
Vine                    91
Twitter Web Client      31
TweetDeck               10
Name: source, dtype: int64

### Issue: urls are duplicated and separated by commas

Define:
* Clean the expanded urls with NaN 
* replace the expanded url with the tweet url - https://twitter.com/dog_rates/status/{tweet_id}

**Code:**

In [85]:
tweet_ids = tweet_stats[tweet_stats['expanded_urls'].isna()]['tweet_id'].to_list()

In [86]:
dog_ratings[dog_ratings['tweet_id'].isin(tweet_ids)]

dog_id            tweet_id  \
375      296  828361771580813312   
707      539  785515384317313025   
1445    1223  696518437233913856   

                                                           text  \
375   Beebop and Doobert should start a band 12/10 would listen   
707              Today, 10/10, should be National Dog Rates Day   
1445            Oh my god 10/10 for every little hot dog pupper   

      rating_numerator  rating_denominator  name dog_type  
375                 12                  10  None     None  
707                 10                  10  None     None  
1445                10                  10  None   pupper

In [87]:
# these aren't dog ratings so drop them from each df
dog_ratings.drop(dog_ratings[dog_ratings['tweet_id'].isin(tweet_ids)].index, axis=0, inplace=True)
tweet_stats.drop(tweet_stats[tweet_stats['tweet_id'].isin(tweet_ids)].index, axis=0, inplace=True)
df_ip_clean.drop(df_ip_clean[df_ip_clean['tweet_id'].isin(tweet_ids)].index, axis=0, inplace=True)

In [88]:
# replace expanded urls with the tweet url
tweet_stats['tweet_url'] = tweet_stats['tweet_id'].apply(lambda x: "https://twitter.com/dog_rates/status/" + str(x))

In [89]:
# drop the original expanded urls column 
tweet_stats.drop(columns=['expanded_urls'], inplace=True)

**Test:**

In [90]:
# confirm this worked
tweet_stats.sample(5)

tweet_id  retweet_count  reply_count  like_count  quote_count  \
937   725458796924002305          557.0         23.0      1301.0          0.0   
804   747103485104099331         3620.0         67.0      8866.0        146.0   
1701  673680198160809984          416.0         19.0      1227.0          1.0   
428   805487436403003392         2373.0         35.0      8337.0        100.0   
1923  669015743032369152          323.0          7.0       648.0          0.0   

                      timestamp              source  \
937   2016-04-27 22:57:10 +0000  Twitter for iPhone   
804   2016-06-26 16:25:26 +0000  Twitter for iPhone   
1701  2015-12-07 01:47:30 +0000  Twitter for iPhone   
428   2016-12-04 19:02:24 +0000  Twitter for iPhone   
1923  2015-11-24 04:52:37 +0000  Twitter for iPhone   

                                                    tweet_url  
937   https://twitter.com/dog_rates/status/725458796924002305  
804   https://twitter.com/dog_rates/status/747103485104099331  
1701  https://twitter.com/dog_rates/status/673680198160809984  
428   https://twitter.com/dog_rates/status/805487436403003392  
1923  https://twitter.com/dog_rates/status/669015743032369152

### Issue: Timestamp is a string, not a datetime object

Define: 
* use `pd.to_datetime()` convert the series

**Code:**

In [91]:
# convert datetime string to datetime object
tweet_stats['timestamp'] = pd.to_datetime(tweet_stats['timestamp'])

**Test:**

In [92]:
# confirm this worked
tweet_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2086 entries, 0 to 2096
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   tweet_id       2086 non-null   int64              
 1   retweet_count  2086 non-null   float64            
 2   reply_count    2086 non-null   float64            
 3   like_count     2086 non-null   float64            
 4   quote_count    2086 non-null   float64            
 5   timestamp      2086 non-null   datetime64[ns, UTC]
 6   source         2086 non-null   category           
 7   tweet_url      2086 non-null   object             
dtypes: category(1), datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 132.6+ KB


### Issue: Tweet 810984652412424192 should be dropped - not a rating

**Define:**
* drop tweet_id 810984652412424192 from each df by filtering

**Code:**

In [93]:
dog_ratings = dog_ratings[dog_ratings['tweet_id']!=666287406224695296]
tweet_stats = tweet_stats[tweet_stats['tweet_id']!=666287406224695296]
df_ip_clean = df_ip_clean[df_ip_clean['tweet_id']!=666287406224695296]

**Test:**

In [94]:
(dog_ratings['tweet_id']==666287406224695296).sum() + \
(tweet_stats['tweet_id']==666287406224695296).sum() + \
(df_ip_clean['tweet_id']==666287406224695296).sum()

0

### Issue: Tweet 666287406224695296 rating should be corrected to 9/10 (incorrectly picked up as 24/7)

**Define:**
* Set the values of `rating_numerator` and `rating_denominator` using `.loc`

**Code:**

In [95]:
dog_ratings.loc[dog_ratings['tweet_id']==666287406224695296, ['rating_numerator', 'rating_denominator']] = [9,10]

In [96]:
# also the name is wrong, so fix that while we're on it
dog_ratings.loc[dog_ratings['tweet_id']==666287406224695296, 'name'] = 'None'

**Test:**

In [97]:
dog_ratings[dog_ratings['tweet_id']==666287406224695296]

Empty DataFrame
Columns: [dog_id, tweet_id, text, rating_numerator, rating_denominator, name, dog_type]
Index: []

## Storing data

In [98]:
if not os.path.exists('./output'):
    os.makedirs('./output/')

# output data to csvs
dog_ratings.to_csv('./output/twitter_archive_master.csv', index=False)
tweet_stats.to_csv('./output/twitter_archive_stats.csv', index=False)
df_ip_clean.to_csv('./output/twitter_archive_image_predictions.csv', index=False)

In [99]:
# read in the data and confirm it looks right
dog_ratings= pd.read_csv('./output/twitter_archive_master.csv')
tweet_stats= pd.read_csv('./output/twitter_archive_stats.csv')
df_ip_clean= pd.read_csv('./output/twitter_archive_image_predictions.csv')

In [100]:
dog_ratings.head()

dog_id            tweet_id  \
0       1  892420643555336193   
1       2  892177421306343426   
2       3  891815181378084864   
3       4  891689557279858688   
4       5  891327558926688256   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

   rating_numerator  rating_denominator      name dog_type  
0                13                  10   Phineas     None  
1                13                  10     Tilly     None  
2                12                  10    Archie     None  
3                13                  10     Darla     None  
4                12                  10  Franklin     None

In [101]:
tweet_stats.head()

tweet_id  retweet_count  reply_count  like_count  quote_count  \
0  892420643555336193         6982.0        139.0     33729.0        219.0   
1  892177421306343426         5284.0        170.0     29261.0        274.0   
2  891815181378084864         3469.0        110.0     21996.0        145.0   
3  891689557279858688         7197.0        142.0     36819.0        286.0   
4  891327558926688256         7731.0        193.0     35199.0        296.0   

                   timestamp              source  \
0  2017-08-01 16:23:56+00:00  Twitter for iPhone   
1  2017-08-01 00:17:27+00:00  Twitter for iPhone   
2  2017-07-31 00:18:03+00:00  Twitter for iPhone   
3  2017-07-30 15:58:51+00:00  Twitter for iPhone   
4  2017-07-29 16:00:24+00:00  Twitter for iPhone   

                                                 tweet_url  
0  https://twitter.com/dog_rates/status/892420643555336193  
1  https://twitter.com/dog_rates/status/892177421306343426  
2  https://twitter.com/dog_rates/status/891815181378084864  
3  https://twitter.com/dog_rates/status/891689557279858688  
4  https://twitter.com/dog_rates/status/891327558926688256

In [102]:
df_ip_clean.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

## Analyzing and Visualizing Data

### Question: How are ratings distributed?

In [103]:
# do a cut into 0-4, 5-9, 10-14, 15-19, 20+
dog_ratings['score_range'] = pd.cut(dog_ratings['rating_numerator'], bins=[0, 4, 9, 14, 19, 2000], labels=['0-4', '5-9', '10-14', '15-19', '20+'])

In [104]:
# confirm this worked
dog_ratings.sample(5)

dog_id            tweet_id  \
1051    1054  709409458133323776   
1278    1282  691416866452082688   
1688    1692  673711475735838725   
332      334  821522889702862852   
2078    2083  666094000022159362   

                                                                                                                                               text  \
1051                                                                This is Billy. He sensed a squirrel. 8/10 damn it Billy https://t.co/Yu0K98VZ9A   
1278                                    I present to you... Dog Jesus. 13/10 (he could be sitting on a rock but I doubt it) https://t.co/fR1P3g5I6k   
1688                                                                             🎶 HELLO FROM THE OTHER SIIIIIIIIDE 🎶 10/10 https://t.co/MTOOksRzvH   
332   This is Harlso. He has a really good idea but isn't sure you're going to like it. 13/10 he'll just keep it to himself https://t.co/IzcaR3Nqyn   
2078           This appears to be a Mongolian Presbyterian mix. Very tired. Tongue slip confirmed. 9/10 would lie down with https://t.co/mnioXo3IfP   

      rating_numerator  rating_denominator    name dog_type score_range  
1051                 8                  10   Billy     None         5-9  
1278                13                  10    None     None       10-14  
1688                10                  10    None     None       10-14  
332                 13                  10  Harlso     None       10-14  
2078                 9                  10    None     None         5-9

In [105]:
# get the % splits of dog ratings
dog_ratings.groupby('score_range')['tweet_id'].count()/dog_ratings['tweet_id'].count()

score_range
0-4      0.022934
5-9      0.176302
10-14    0.791687
15-19    0.000000
20+      0.008600
Name: tweet_id, dtype: float64

In [106]:
# define function for plotting a (vertical) column chart with relative y axis
def rel_col_chart(chart_data, xlabel=None, ylabel=None, title=None, plot_order=None):
    base_color = sns.color_palette()[0]
    plt.figure(figsize=[8,5])

    sns.countplot(x=chart_data, color=base_color, order=plot_order);

    # set up y axis which goes up to 100%
    y_ticks_max = chart_data.value_counts().max() / chart_data.count()
    if y_ticks_max > 0.5:
        y_tick_step = 0.1
    else:
        y_tick_step = 0.05
    y_tick_props = np.arange(0, y_ticks_max, y_tick_step) 
    y_tick_names = ['{:0.0f}%'.format(v * 100) for v in y_tick_props]

    # set x and y labels
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    ax = plt.gca()
    ax.set_yticks(y_tick_props * chart_data.count(), labels=y_tick_names)
    sns.despine();


In [107]:
# plot a relative column chart
rel_col_chart(
    chart_data = dog_ratings['score_range'],
    title = 'Percentage of ratings by numerator',
    xlabel = 'Rating numerator',
    ylabel = '% of ratings'
)

**Observations:**
* Over 70% of ratings are positive, in the 10-14 range
* Very few ratings are over 15
* A significant proportion of ratings (~20%) are negative, i.e. less than 10

### Question: What are the most common dog names?

In [108]:
# count the names (excluding none) and filter for the 10 most frequent
dog_ratings.groupby('name')['dog_id'].count().drop('None').nlargest(10)

name
Charlie    11
Lucy       11
Cooper     10
Oliver     10
Penny       9
Tucker      9
Lola        8
Sadie       8
Winston     8
Daisy       7
Name: dog_id, dtype: int64

**Observations:**
* Charlie and Lucy are the joint most common name, followed by Cooper and Oliver

### Question: What dog breeds are the most common

In [109]:
# join the dog_rating and df_ip_clean datasets
# use inner join to filter out any rows where we don't have an image
df_merged = dog_ratings.merge(right=df_ip_clean, on='tweet_id', how='inner')

In [110]:
# confirm this worked 
df_merged.sample(5)

dog_id            tweet_id  \
507      538  785533386513321988   
1961    2089  666057090499244032   
683      727  754120377874386944   
1149    1245  694001791655137281   
690      735  753026973505581056   

                                                                                                                                              text  \
507                                                        This is Dallas. Her tongue is ridiculous. 11/10 h*ckin proud af https://t.co/h4jhlH4EyG   
1961                  My oh my. This is a rare blond Canadian terrier on wheels. Only $8.98. Rather docile. 9/10 very rare https://t.co/yWBqbrzy8O   
683          When you hear your owner say they need to hatch another egg, but you've already been on 17 walks today. 10/10 https://t.co/lFEoGqZ4oA   
1149  This is Chester. He's a Benefloof Cumberbark. Fabulous ears. Nifty shirt. Was probably on sale. Nice hardwood. 11/10 https://t.co/YoII7tWXMT   
690   Say hello to Tayzie. She's a Barbadian Bugaboop. Seems quite social. A rare quality for a Bugaboop. 10/10 petable af https://t.co/6qF5YZx6OV   

      rating_numerator  rating_denominator     name dog_type score_range  \
507                 11                  10   Dallas     None       10-14   
1961                 9                  10     None     None         5-9   
683                 10                  10     None     None       10-14   
1149                11                  10  Chester     None       10-14   
690                 10                  10   Tayzie     None       10-14   

                                              jpg_url  img_num  \
507   https://pbs.twimg.com/media/CubGchjXEAA6gpw.jpg        2   
1961  https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg        1   
683   https://pbs.twimg.com/media/CncseIzWgAA4ghH.jpg        1   
1149  https://pbs.twimg.com/media/CaGW8JQUMAEVtLl.jpg        1   
690   https://pbs.twimg.com/media/CnNKCKKWEAASCMI.jpg        3   

                      p1   p1_conf  p1_dog                       p2   p2_conf  \
507   miniature_pinscher  0.436023    True  black-and-tan_coonhound  0.258049   
1961       shopping_cart  0.962465   False          shopping_basket  0.014594   
683                 chow  0.168909    True          Norfolk_terrier  0.129114   
1149            Pembroke  0.769999    True                 Cardigan  0.229228   
690             Pembroke  0.868511    True                 Cardigan  0.103708   

      p2_dog                 p3   p3_conf  p3_dog  
507     True         Rottweiler  0.145231    True  
1961   False   golden_retriever  0.007959    True  
683     True         Pomeranian  0.120822    True  
1149    True          Chihuahua  0.000247    True  
690     True  Shetland_sheepdog  0.018142    True

In [111]:
# do a quick filter for photos which are identifiable as dogs only
df_merged = df_merged[df_merged['p1_dog']]

In [112]:
#confirm this worked
df_merged.sample(5)

dog_id            tweet_id  \
857      920  728387165835677696   
1230    1334  688116655151435777   
420      446  801167903437357056   
1544    1670  674269164442398721   
404      428  805520635690676224   

                                                                                                                                                                    text  \
857                            This is Enchilada (yes, that's her real name). She's a Low-Cruisin Plopflopple. Very rare. Only a few left. 12/10 https://t.co/SiaiTWgsfP   
1230                         Please send dogs. I'm tired of seeing other stuff like this dangerous pirate. We only rate dogs. Thank you... 10/10 https://t.co/YdLytdZOqv   
420                                                                                     This is Milo. I would do terrible things for Milo. 13/10 https://t.co/R6wJyC2Tey   
1544                        This is Bob. He's a Juniper Fitzsimmons. His body is 2, but his face is 85. Always looks miserable. Nice stool. 8/10 https://t.co/vYe9RlVz2N   
404   This is Zeke the Wonder Dog. He never let that poor man keep his frisbees. One of the Spartans all time greatest receivers. 13/10 RIP Zeke https://t.co/zacX7S6GyJ   

      rating_numerator  rating_denominator       name dog_type score_range  \
857                 12                  10  Enchilada     None       10-14   
1230                10                  10       None     None       10-14   
420                 13                  10       Milo     None       10-14   
1544                 8                  10        Bob     None         5-9   
404                 13                  10       Zeke     None       10-14   

                                              jpg_url  img_num  \
857   https://pbs.twimg.com/media/ChvAQuMWMAAVaKD.jpg        1   
1230  https://pbs.twimg.com/media/CYyucekVAAESj8K.jpg        1   
420   https://pbs.twimg.com/media/Cx5R8wPVEAALa9r.jpg        1   
1544  https://pbs.twimg.com/media/CVt8OmIWIAAbxvJ.jpg        1   
404   https://pbs.twimg.com/media/Cy3IvdZXgAUoEaj.jpg        1   

                  p1   p1_conf  p1_dog                  p2   p2_conf  p2_dog  \
857           collie  0.266414    True      Great_Pyrenees  0.138546    True   
1230             pug  0.973819    True           Chihuahua  0.010891    True   
420   cocker_spaniel  0.740220    True      Dandie_Dinmont  0.061604    True   
1544             pug  0.622921    True  Norwegian_elkhound  0.048659    True   
404         malinois  0.643147    True     German_shepherd  0.186642    True   

                             p3   p3_conf  p3_dog  
857                    keeshond  0.109014    True  
1230  Staffordshire_bullterrier  0.006864    True  
420              English_setter  0.041331    True  
1544                   Cardigan  0.016966    True  
404              Border_terrier  0.109345    True

In [113]:
# get the 10 highest counts of breed
df_merged.groupby('p1')['tweet_id'].count().nlargest(10)

p1
golden_retriever      137
Labrador_retriever     94
Pembroke               88
Chihuahua              78
pug                    54
chow                   41
Samoyed                40
Pomeranian             38
toy_poodle             37
malamute               29
Name: tweet_id, dtype: int64

In [114]:
# define function for plotting a (horizontal) bar chart with relative x axis
def rel_bar_chart(chart_data, xlabel=None, ylabel=None, title=None, plot_order=[]):
    base_color = sns.color_palette()[0]
    plt.figure(figsize=[8,5])

    # convert plot order to a list if needed
    if isinstance(plot_order, pd.Index):
        plot_order = list(plot_order.values)

#     if plot_order == []:
#         sns.countplot(y=chart_data, color=base_color, order=chart_data.value_counts().index.to_list());
#     else:
#         sns.countplot(y=chart_data, color=base_color, order=plot_order);

    if plot_order == []:
        plot_order = chart_data.value_counts().index.to_list()[:10]

    sns.countplot(y=chart_data, color=base_color, order=plot_order);



    # set up y axis which goes up to 100%
    x_ticks_max = chart_data.value_counts().max() / chart_data.count()
    if x_ticks_max > 0.5:
        x_tick_step = 0.1
    else:
        x_tick_step = 0.05
    x_tick_props = np.arange(0, x_ticks_max, x_tick_step) 
    x_tick_names = ['{:0.0f}%'.format(v * 100) for v in x_tick_props]

    # set x and y labels
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    ax = plt.gca()
    ax.set_xticks(x_tick_props * chart_data.count(), labels=x_tick_names)
    sns.despine();

    # Logic to print the proportion text on the bars
    counts = chart_data.value_counts()
#    if plot_order == []:
#        plot_order = chart_data.value_counts().index.to_list()

    for i in range(len(plot_order)):
        count = counts.loc[plot_order[i]]
        # Convert count into a percentage, and then into string
        pct_string = '{:0.1f}%'.format(100 * count / chart_data.count())
        # Print the string value on the bar. 
        plt.text(count + counts.max()*0.01, i, pct_string, va='center')

In [115]:
# plot this information as a relative bar chart (top 10 categories only)
rel_bar_chart(
    chart_data=df_merged['p1'],
    title='10 dog breeds most frequently rated',
    xlabel='Frequency',
    ylabel='Breed'
)

**Observations:**
* Golden and Labrador retrievers are the most popular categories

### Question: which breeds of dog get the higest rating?

In [116]:
# create a dataframe with the average breed scores and count of the number of ratings
df_breed_scores = pd.DataFrame({
    'ave_rating' : df_merged.groupby('p1')['rating_numerator'].mean(),
    'count_of_ratings': df_merged.groupby('p1')['tweet_id'].count()
}).sort_values('ave_rating', ascending=False)
df_breed_scores.index.names = ['dog_breed']

In [117]:
# confirm this worked
df_breed_scores.head()

ave_rating  count_of_ratings
dog_breed                                                
clumber                       27.000000                 1
soft-coated_wheaten_terrier   26.700000                10
West_Highland_white_terrier   15.642857                14
Great_Pyrenees                14.928571                14
borzoi                        14.625000                 8

In [118]:
# filter for dog's breeds with at least 5 ratings, and then get the top average ratings
df_breed_scores[df_breed_scores['count_of_ratings']>=5].head(10)

ave_rating  count_of_ratings
dog_breed                                                
soft-coated_wheaten_terrier   26.700000                10
West_Highland_white_terrier   15.642857                14
Great_Pyrenees                14.928571                14
borzoi                        14.625000                 8
Labrador_retriever            13.936170                94
Siberian_husky                13.250000                20
Pomeranian                    12.868421                38
golden_retriever              12.277372               137
Border_terrier                12.142857                 7
kuvasz                        12.000000                15

**Observations:**
* The soft-coated wheaten terrier get the highest average ratings, and 
* With an average score of 26.7, the soft-coated wheaten is far ahead of the 2nd most highly ranked category, at 15.6  